In [4]:
import requests
import json

In [6]:
def fetch_item_price(item_id):
    """
    Fetches the price of an item from the Old School RuneScape Grand Exchange API.

    Args:
        item_id (int): The ID of the item to fetch.

    Returns:
        dict: A dictionary containing the item's name and current price, or None if the request fails.
    """
    url = "https://prices.runescape.wiki/api/v1/osrs/1h"

    try:
        response = requests.get(url, headers={
        'User-Agent': '@el on discord'
        })
        response.raise_for_status()  # Raise an HTTPError for bad responses (4xx and 5xx)

        data = response.json()
        item_data = data.get("data", {}).get(str(item_id), {})

        if not item_data:
            print(f"Item ID {item_id} not found in the API response.")
            return None

        return {
            "id": item_id,
            "high": item_data.get("high"),
            "low": item_data.get("low")
        }
    except requests.exceptions.RequestException as e:
        print(f"Error fetching data for item ID {item_id}: {e}")
        return None

In [8]:
print("Old School RuneScape Grand Exchange Price Scraper")

# Example: List of item IDs to fetch
item_ids = [4151, 11840, 12934]  # Abyssal whip, Dragon boots, Zulrah's scales

for item_id in item_ids:
    item_data = fetch_item_price(item_id)

    if item_data:
        print(f"Item ID: {item_data['id']} - High Price: {item_data['avgHighPrice']} - Low Price: {item_data['avgLowPrice']}")
    else:
        print(f"Failed to fetch data for item ID {item_id}")

Old School RuneScape Grand Exchange Price Scraper
Item ID: 4151 - High Price: None - Low Price: None
Item ID: 11840 - High Price: None - Low Price: None
Item ID: 12934 - High Price: None - Low Price: None
